# **Installing Requirements**

Since the project is developed by different people, we will install all the requirements using the requirements.txt file which specifies all the packets' version that must be installed.

In [ ]:
%pip install -r ../requirements.txt

# **Downloading Files from GDrive**

Access denied with the following error:



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?export=download&confirm=pbef&id=1lF3pmEZvaQNvoV5bi7FQreS6l-iFU6J6 



# **Face Recognition**

Face recognition is a computer vision task that involves identifying and verifying a person's identity based on their facial features. This process can be broken down into these steps:

1. **Detection**: Identifying faces in images or video frames.
2. **Feature** Extraction: Capturing unique facial characteristics.
3. **Representation**: Creating a distinctive template for each face.
4. **Model Training**: Associating templates with known identities during training.
5. **Matching**: Comparing a new face's template to stored ones for identification.
6. **Decision**: Determining a match based on a similarity threshold.

Nowadays, these steps are performed through deep learning models. In the following section we will provide a simple implementation through a pre-trained model and our paper implementation (further details in the next sections).

## **Load all the user faces**

All the registered users' face are saved into the UserFaces folder. We have to build our "dataset" of faces on which the recognition will be performed.

In [ ]:
import os
import cv2
import dlib

detector = dlib.get_frontal_face_detector() # Load the pre-trained face detector from dlib
face_recognizer = dlib.shape_predictor("../Models/shape_predictor_68_face_landmarks.dat") # Load the pre-trained face recognition model 

# Initialize lists to store known faces and their corresponding names
known_faces = []
known_names = []

base_directory = "../UserFaces/" # Directory containing user faces

# Iterate through directories
for user_directory in os.listdir(base_directory):

    user_path = os.path.join(base_directory, user_directory)

    # Iterate through face images in each user directory
    for filename in os.listdir(user_path):
        image_path = os.path.join(user_path, filename)

        img = cv2.imread(image_path) # Read the image
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Convert it into a gray scale image

        faces = detector(gray) # Detect faces in the image (there will be only one face)

        if faces:
            face_landmarks = face_recognizer(img, faces[0]) # Obtain the landmarks

            # Add face landmarks and user name to the lists
            known_faces.append(face_landmarks)
            known_names.append(user_directory)

print(known_names)

## **Pre-trained Model**

This model is provided by the dlib library. Dlib's face recognition model is based on classical machine learning techniques rather than deep learning. It uses a combination of HOG (Histogram of Oriented Gradients) features and a SVM (Support Vector Machine) classifier to identify and recognize faces.

In [ ]:
cap = cv2.VideoCapture(0) # Open a connection to the webcam (0 represents the default camera)

while True:
    ret, frame = cap.read() # Read a frame from the webcam

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # Convert the frame to grayscale for face detection
    
    faces = detector(gray) # Detect faces in the frame

    # Iterate over detected faces
    for face in faces:
        landmarks = face_recognizer(frame, face) # Get facial landmarks

        # Compare with known faces
        for i, known_face in enumerate(known_faces):

            if landmarks == known_face:
                name = known_names[i]
                print(f"Hello, {name}!")

        # Draw a rectangle around the face
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display the result
    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'): # Break the loop if the 'q' key is pressed
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()
